In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
from tabulate import tabulate
import pyarrow as pa
import pyarrow.parquet as pq
import json
import pickle as pkl
import plotly.graph_objects as go
from plotting import CandlePlot
pd.set_option("display.max_columns", None)

In [2]:
class Data:
    
    def __init__(self, source):
        assert type(source) == str or type(source) == pd.DataFrame, 'Invalid source'
        if type(source) == str:
            self.df = {
                'raw': pd.read_pickle(source)
            }
        elif type(source) == pd.DataFrame:
            self.df = {
                'raw': source.copy()
            }            

        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]
        self.datalen = self.df['raw'].shape[0]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(pd.concat([df.head(2), df.tail(1)])) + '\n'
        return repr

    def prep_data(self, name: str, start: int, end: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows as preparation for fast data creation
        '''
        # assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'

        if cols == None:
            cols = self.df[source].columns
        # if direction == 1:
        #     self.df[name] = self.df[source][cols].iloc[:rows].copy()
        # else:
        #     self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        if start == None:
            start = 0
        if end == None:
            end = self.datalen

        assert end > start, f'start={start}, end={end} not valid'

        self.df[name] = self.df[source][cols].iloc[start:end].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: dict):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        # cols = exist_cols + cols
        for col, _type in cols.items():
            self.df[name][col] = pd.Series(dtype=_type) #self.df[name][col].apply(_type)
        # self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str, start: int, end: int, source: str='raw', cols: list=None, add_cols: dict=None):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fastdf = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata()
        '''

        self.prep_data(name=name, start=start, end=end, source=source, cols=cols)
        if add_cols:
            # types = dict() if types is None else types
            self.add_columns(name=name, cols=add_cols)

        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]
        self.fdatalen = len(self.fastdf[0])

    def fdata(self, column: str=None, index: int=None, rows: int=None):
        if column is None:
            return self.fastdf
        if index is None:
            return self.fastdf[self.fcols[column]]
        else:
            if rows:
                try:
                    return self.fastdf[self.fcols[column]][index:index+rows]
                except:
                    return self.fastdf[self.fcols[column]][index:]
            else:
                return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, index: int=None, value: float=None):
        assert value is not None, 'Value cannot be null'
        if index is None:
            assert len(value) == self.fdatalen
            for i in range(self.fdatalen):
                self.fastdf[self.fcols[column]][i] = value[i]
                print(i, )
        else:
            self.fastdf[self.fcols[column]][index] = value

In [3]:
with open("../data/instruments.json", 'r') as f:
    instr = json.load(f)

In [4]:
ticker = 'EUR_GBP'

In [5]:
instr[ticker]

{'name': 'EUR_GBP',
 'type': 'CURRENCY',
 'displayName': 'EUR/GBP',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1',
 'marginRate': '0.03333333333333'}

In [6]:
# df = pd.read_pickle(f"../data/{ticker}_M5.pkl")

In [7]:
# df['pip_return'] = (df.mid_c - df.mid_c.shift(1)) * pow(10, -instr[ticker]['pipLocation'])

In [8]:
# pd.concat([df.head(3), df.tail(3)])

In [9]:
# our_cols = ['time', 'mid_c', 'mid_c', 'bid_c', 'ask_c']

In [10]:
# d = Data(df[our_cols])
# d

In [11]:
SIZE, ENTRY, TP, SL = 0, 1, 2, 3
EXIT, PIPS = 2, 3

In [12]:
def read_data(ticker: str, frequency: str):
    our_cols = ['time', 'mid_c', 'bid_c', 'ask_c']
    df = pd.read_pickle(f"../data/{ticker}_{frequency}.pkl")
    return df[our_cols]

In [13]:
def cum_long_position(d: Data, i: int):
    open_longs = d.fdata('open_longs', i).copy() if type(d.fdata('open_longs', i)) == dict else dict()
    cum_long_position = 0
    for _, trade in open_longs.items():
        cum_long_position = cum_long_position + trade[SIZE]
    return cum_long_position

def cum_short_position(d: Data, i: int):
    open_shorts = d.fdata('open_shorts', i).copy() if type(d.fdata('open_shorts', i)) == dict else dict()
    cum_short_position = 0
    for _, trade in open_shorts.items():
        cum_short_position = cum_short_position + trade[SIZE]
    return cum_short_position

def unrealised_pnl(d: Data, i: int):
    open_longs = d.fdata('open_longs', i).copy() if type(d.fdata('open_longs', i)) == dict else dict()
    open_shorts = d.fdata('open_shorts', i).copy() if type(d.fdata('open_shorts', i)) == dict else dict()
    pnl = 0
    for _, trade in open_longs.items():
        pnl = pnl + trade[SIZE] * (d.fdata('mid_c', i) - trade[ENTRY])
    for _, trade in open_shorts.items():
        pnl = pnl + trade[SIZE] * (trade[ENTRY] - d.fdata('mid_c', i))
    return round(pnl, 2)

def realised_pnl(d: Data, i: int):
    closed_longs = d.fdata('closed_longs', i).copy() if type(d.fdata('closed_longs', i)) == dict else dict()
    closed_shorts = d.fdata('closed_shorts', i).copy() if type(d.fdata('closed_shorts', i)) == dict else dict()
    pnl = 0
    for _, trade in closed_longs.items():
        pnl = pnl + trade[SIZE] * (trade[EXIT] - trade[ENTRY])
    for _, trade in closed_shorts.items():
        pnl = pnl + trade[SIZE] * (trade[ENTRY] - trade[EXIT])
    return round(pnl, 2)

def unrealised_pnl_prev_bar(d: Data, i: int):
    open_longs = d.fdata('open_longs', i-1).copy() if type(d.fdata('open_longs', i)) == dict else dict()
    open_shorts = d.fdata('open_shorts', i-1).copy() if type(d.fdata('open_shorts', i)) == dict else dict()
    pnl = 0
    for _, trade in open_longs.items():
        pnl = pnl + trade[SIZE] * (d.fdata('mid_c', i) - trade[ENTRY])
    for _, trade in open_shorts.items():
        pnl = pnl + trade[SIZE] * (trade[ENTRY] - d.fdata('mid_c', i))
    return round(pnl, 2)

In [14]:
def dynamic_trade_size(d: Data, i: int, sizing_ratio: float):
    # _ac_bal = d.fdata('ac_bal', i-1) + d.fdata('realised_pnl', i)
    _net_bal = d.fdata('ac_bal', i-1) + unrealised_pnl_prev_bar(d, i)
    return int(_net_bal * sizing_ratio)   

In [15]:
def open_trades(d: Data, i: int, tp_pips: int, sl_pips: int, init_trade_size: int, sizing: str, sizing_ratio: float, trade_no: int):
    long_tp = d.fdata('mid_c', i) + tp_pips * pow(10, instr[ticker]['pipLocation'])
    short_tp = d.fdata('mid_c', i) - tp_pips * pow(10, instr[ticker]['pipLocation'])

    long_sl = d.fdata('mid_c', i) - sl_pips * pow(10, instr[ticker]['pipLocation'])
    short_sl = d.fdata('mid_c', i) + sl_pips * pow(10, instr[ticker]['pipLocation'])

    if i == 0:
        open_longs, open_shorts = dict(), dict()
        trade_size = init_trade_size
    else:
        open_longs, open_shorts = d.fdata('open_longs', i-1).copy(), d.fdata('open_shorts', i-1).copy()
        # Temporary data population for dynamic trade size calculation
        d.update_fdata('open_longs', i, open_longs)
        d.update_fdata('open_shorts', i, open_shorts)
        trade_size = init_trade_size if sizing == 'static' else dynamic_trade_size(d, i, sizing_ratio)

    open_longs[trade_no] = (trade_size, d.fdata('ask_c', i), long_tp, long_sl) # (SIZE, ENTRY, TP, SL)
    d.update_fdata('open_longs', i, open_longs)
    # d.update_fdata('open_longs', i, str(open_longs).replace("'",'"'))
    open_shorts[trade_no] = (trade_size, d.fdata('bid_c', i), short_tp, short_sl) # (SIZE, ENTRY, TP, SL)
    d.update_fdata('open_shorts', i, open_shorts)
    # d.update_fdata('open_shorts', i, str(open_shorts).replace("'",'"'))

    if d.fdata('trade_type', i) != 0: # Do not update if stop loss was triggered on this bar
        d.update_fdata('trade_type', i, 1)
    
    return long_tp, short_tp # equals next_up_grid, next_down_grid for next grid level trades

In [16]:
def close_long(d: Data, i: int, trade_no: int, sl=False):
    # Remove from open longs
    open_longs = d.fdata('open_longs', i).copy()
    closing_long = open_longs[trade_no]
    del open_longs[trade_no]
    d.update_fdata('open_longs', i, open_longs)

    # Append to closed longs
    pips = (d.fdata('ask_c', i) - closing_long[ENTRY]) * pow(10, -instr[ticker]['pipLocation'])
    # if type(d.fdata('closed_longs', i)) != dict:
    #     closed_longs = dict()
    # else:
    #     closed_longs = d.fdata('closed_longs', i).copy()
    closed_longs = d.fdata('closed_longs', i).copy() if type(d.fdata('closed_longs', i)) == dict else dict()
    closed_longs[trade_no] = (closing_long[SIZE], closing_long[ENTRY], d.fdata('bid_c', i), round(pips, 1)) # (SIZE, ENTRY, EXIT, PIPS)

    d.update_fdata('closed_longs', i, closed_longs)
    if sl == True:
        d.update_fdata('trade_type', i, 0)
    else:
        d.update_fdata('trade_type', i, 1)

In [17]:
def close_short(d: Data, i: int, trade_no: int, sl=False):
    # Remove from open shorts
    open_shorts = d.fdata('open_shorts', i).copy()
    closing_short = open_shorts[trade_no]
    del open_shorts[trade_no]
    d.update_fdata('open_shorts', i, open_shorts)

    # Append to closed shorts
    pips = (closing_short[ENTRY] - d.fdata('bid_c', i)) * pow(10, -instr[ticker]['pipLocation'])
    # if type(d.fdata('closed_shorts', i)) != dict:
    #     closed_shorts = dict()
    # else:
    #     closed_shorts = d.fdata('closed_shorts', i).copy()
    closed_shorts = d.fdata('closed_shorts', i).copy() if type(d.fdata('closed_shorts', i)) == dict else dict()
    closed_shorts[trade_no] = (closing_short[SIZE], closing_short[ENTRY], d.fdata('ask_c', i), round(pips, 1)) # (SIZE, ENTRY, EXIT, PIPS)

    d.update_fdata('closed_shorts', i, closed_shorts)
    if sl == True:
        d.update_fdata('trade_type', i, 0)
    else:
        d.update_fdata('trade_type', i, 1)

In [18]:
def current_values(d, i):
    _cum_long_position = cum_long_position(d, i)
    _cum_short_position = cum_short_position(d, i)
    _unrealised_pnl = unrealised_pnl(d, i)
    _realised_pnl = realised_pnl(d, i)
    ac_bal = d.fdata('ac_bal', i-1) + _realised_pnl
    margin_used = (_cum_long_position + _cum_short_position) * float(instr[ticker]['marginRate'])
    margin_closeout = ac_bal + _unrealised_pnl
    return margin_used, margin_closeout

def stop_loss_oldest(d: Data, i: int, margin_sl_percent: float=0.5):
    margin_used, margin_closeout = current_values(d, i)
    # Margin call is triggered when margin closeout is less than 50% of margin used.
    # Here stop loss is triggered if it falls below margin_sl_percent
    if margin_closeout < margin_used * margin_sl_percent:
        reduced_margin = margin_closeout / margin_sl_percent
        while reduced_margin < margin_used:
            longs = list(d.fdata('open_longs', i).keys())
            shorts = list(d.fdata('open_shorts', i).keys())
            oldest_long = longs[0] if len(longs) > 0 else None
            oldest_short = shorts[0] if len(shorts) > 0 else None
            if oldest_long == None and oldest_short == None:
                break
            elif oldest_long == None:
                close_short(d, i, oldest_short, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, short: {oldest_short}')
            elif oldest_short == None:
                close_long(d, i, oldest_long, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, long: {oldest_long}')
            else:
                if oldest_long <= oldest_short:
                    close_long(d, i, oldest_long, sl=True)
                    margin_used, _ = current_values(d, i)
                    # print(f'stop loss: {i}, long: {oldest_long}')
                else:
                    close_short(d, i, oldest_short, sl=True)
                    margin_used, _ = current_values(d, i)
                    # print(f'stop loss: {i}, short: {oldest_short}')

def stop_loss_farthest(d: Data, i: int, margin_sl_percent: float):
    margin_used, margin_closeout = current_values(d, i)
    # Margin call is triggered when margin closeout is less than 50% of margin used.
    # Here stop loss is triggered if it falls below margin_sl_percent
    price = d.fdata('mid_c', i)
    if margin_closeout < margin_used * margin_sl_percent:
        reduced_margin = margin_closeout / margin_sl_percent
        while reduced_margin < margin_used:
            farthest_long_price, farthest_short_price = price, price
            farthest_long, farthest_short = None, None
            for long, trade in d.fdata('open_longs', i).items():
                if trade[ENTRY] > farthest_long_price:
                    farthest_long_price = trade[ENTRY]
                    farthest_long = long
            for short, trade in d.fdata('open_shorts', i).items():
                if trade[ENTRY] < farthest_short_price:
                    farthest_short_price = trade[ENTRY]
                    farthest_short = short
            if farthest_long == None and farthest_short == None:
                break
            elif farthest_long == None:
                close_short(d, i, farthest_short, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, short: {farthest_short}')
            elif farthest_short == None:
                close_long(d, i, farthest_long, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, long: {farthest_long}')
            else:
                if farthest_long_price - price > price - farthest_short_price:
                    close_long(d, i, farthest_long, sl=True)
                    margin_used, _ = current_values(d, i)
                    # print(f'stop loss: {i}, long: {farthest_long}')
                else:
                    close_short(d, i, farthest_short, sl=True)
                    margin_used, _ = current_values(d, i)
                    # print(f'stop loss: {i}, short: {farthest_short}')

def stop_loss_oldest_1(d: Data, i: int, margin_sl_percent: float=0.5):
    margin_used, margin_closeout = current_values(d, i)
    # Margin call is triggered when margin closeout is less than 50% of margin used.
    # Here stop loss is triggered if it falls below margin_sl_percent
    if margin_closeout < margin_used * margin_sl_percent:
        longs = list(d.fdata('open_longs', i).keys())
        shorts = list(d.fdata('open_shorts', i).keys())
        oldest_long = longs[0] if len(longs) > 0 else None
        oldest_short = shorts[0] if len(shorts) > 0 else None
        if oldest_long == None and oldest_short == None:
            pass
        elif oldest_long == None:
            close_short(d, i, oldest_short, sl=True)
            margin_used, _ = current_values(d, i)
            # print(f'stop loss: {i}, short: {oldest_short}')
        elif oldest_short == None:
            close_long(d, i, oldest_long, sl=True)
            margin_used, _ = current_values(d, i)
            # print(f'stop loss: {i}, long: {oldest_long}')
        else:
            if oldest_long <= oldest_short:
                close_long(d, i, oldest_long, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, long: {oldest_long}')
            else:
                close_short(d, i, oldest_short, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, short: {oldest_short}')

def stop_loss_farthest_1(d: Data, i: int, margin_sl_percent: float):
    margin_used, margin_closeout = current_values(d, i)
    # Margin call is triggered when margin closeout is less than 50% of margin used.
    # Here stop loss is triggered if it falls below margin_sl_percent
    price = d.fdata('mid_c', i)
    if margin_closeout < margin_used * margin_sl_percent:
        farthest_long_price, farthest_short_price = price, price
        farthest_long, farthest_short = None, None
        for long, trade in d.fdata('open_longs', i).items():
            if trade[ENTRY] > farthest_long_price:
                farthest_long_price = trade[ENTRY]
                farthest_long = long
        for short, trade in d.fdata('open_shorts', i).items():
            if trade[ENTRY] < farthest_short_price:
                farthest_short_price = trade[ENTRY]
                farthest_short = short
        if farthest_long == None and farthest_short == None:
            pass
        elif farthest_long == None:
            close_short(d, i, farthest_short, sl=True)
            margin_used, _ = current_values(d, i)
            # print(f'stop loss: {i}, short: {farthest_short}')
        elif farthest_short == None:
            close_long(d, i, farthest_long, sl=True)
            margin_used, _ = current_values(d, i)
            # print(f'stop loss: {i}, long: {farthest_long}')
        else:
            if farthest_long_price - price > price - farthest_short_price:
                close_long(d, i, farthest_long, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, long: {farthest_long}')
            else:
                close_short(d, i, farthest_short, sl=True)
                margin_used, _ = current_values(d, i)
                # print(f'stop loss: {i}, short: {farthest_short}')

In [19]:
def calc_ac_values(d: Data, i: int, init_bal: float=None):
    d.update_fdata('cum_long_position', i, cum_long_position(d, i))
    d.update_fdata('cum_short_position', i, cum_short_position(d, i))
    d.update_fdata('unrealised_pnl', i, unrealised_pnl(d, i))
    d.update_fdata('realised_pnl', i, realised_pnl(d, i))
    # First candle
    if i == 0:               
        d.update_fdata('ac_bal', i, init_bal)
    # Subsequent candles
    else:
        d.update_fdata('ac_bal', i, round(d.fdata('ac_bal', i-1) + d.fdata('realised_pnl', i), 2))
    d.update_fdata('margin_used', i, \
                round((d.fdata('cum_long_position', i) + d.fdata('cum_short_position', i)) * float(instr[ticker]['marginRate']), 2))
    d.update_fdata('margin_closeout', i, round(d.fdata('ac_bal', i) + d.fdata('unrealised_pnl', i), 2))

In [20]:
def run_sim(d: Data, sim_name: str, start: int, end: int, init_bal: int, init_trade_size: int, grid_pips: int, 
            sl_grid_count: int, stop_loss: str, margin_sl_percent: float, sizing: str) -> pd.DataFrame:
    tp_pips = grid_pips
    sl_pips = grid_pips * sl_grid_count
    sizing_ratio = init_trade_size / init_bal
    
    add_cols = dict(
        open_longs=object,
        open_shorts=object,
        closed_longs=object,
        closed_shorts=object,
        trade_type=int, # (1 = ENTRY or TP, 0 = SL (ENTRY & TP can also happen when SL is triggered but SL indicator is set))
        cum_long_position=int,
        cum_short_position=int,
        unrealised_pnl=float,
        realised_pnl=float,
        ac_bal=float,
        margin_used=float,
        margin_closeout=float
    )
    d.prepare_fast_data(name=sim_name, start=start, end=end, add_cols=add_cols)

    for i in tqdm(range(d.fdatalen), desc=" Simulating... "):       
        # First candle
        if i == 0:
            # Open new trades
            trade_no = 1            
            next_up_grid, next_down_grid = open_trades(d=d, 
                                                       i=i, 
                                                       tp_pips=tp_pips,
                                                       sl_pips=sl_pips, 
                                                       init_trade_size=init_trade_size, 
                                                       sizing=sizing,
                                                       sizing_ratio=sizing_ratio,
                                                       trade_no=trade_no)
            calc_ac_values(d, i, init_bal)
        # Subsequent candles
        else:
            # Open new trades
            if d.fdata('mid_c', i) >= next_up_grid or d.fdata('mid_c', i) <= next_down_grid:
                trade_no = trade_no + 1            
                next_up_grid, next_down_grid = open_trades(d=d, 
                                                           i=i, 
                                                           tp_pips=tp_pips, 
                                                           sl_pips=sl_pips, 
                                                           init_trade_size=init_trade_size, 
                                                           sizing=sizing,
                                                           sizing_ratio=sizing_ratio,
                                                           trade_no=trade_no)
            else: # Cascade open positions from prev candle to current candle
                d.update_fdata('open_longs', i, d.fdata('open_longs', i-1).copy())
                d.update_fdata('open_shorts', i, d.fdata('open_shorts', i-1).copy())
                
            # Close long positions take profit
            open_longs = d.fdata('open_longs', i).copy()
            for trade_no, trade in open_longs.items():
                if d.fdata('mid_c', i) >= trade[TP]:
                    close_long(d, i, trade_no)

            # Close short positions take profit
            open_shorts = d.fdata('open_shorts', i).copy()
            for trade_no, trade in open_shorts.items():
                if d.fdata('mid_c', i) <= trade[TP]:
                    close_short(d, i, trade_no)

            # Stop loss for grid_count
            if stop_loss == 'grid_count':
                open_longs = d.fdata('open_longs', i).copy()
                for trade_no, trade in open_longs.items():
                    if d.fdata('mid_c', i) <= trade[SL]:
                        close_long(d, i, trade_no, sl=True)

                open_shorts = d.fdata('open_shorts', i).copy()
                for trade_no, trade in open_shorts.items():
                    if d.fdata('mid_c', i) >= trade[SL]:
                        close_short(d, i, trade_no, sl=True)
        
            # Stop loss for margin closeout
            if stop_loss == 'margin_closeout_oldest':
                stop_loss_oldest(d, i, margin_sl_percent)
            elif stop_loss == 'margin_closeout_farthest':
                stop_loss_farthest(d, i, margin_sl_percent)
            elif stop_loss == 'margin_closeout_oldest_1':
                stop_loss_oldest_1(d, i, margin_sl_percent)
            elif stop_loss == 'margin_closeout_farthest_1':
                stop_loss_farthest_1(d, i, margin_sl_percent)

            calc_ac_values(d, i)

    result = d.df[sim_name].copy()
    result = result[(result['trade_type'] == 1) | (result['trade_type'] == 0)]
    del d.df[sim_name]

    return dict(
        sim_name = sim_name,
        init_trade_size=init_trade_size,
        grid_pips=grid_pips,
        sl_grid_count=sl_grid_count,
        stop_loss=stop_loss,
        margin_sl_percent=margin_sl_percent,
        sizing=sizing,
        result = result
    )

In [21]:
def process_sim(d: Data, ticker: str, frequency: str, counter: int, start: int, end:int, init_bal: int, init_trade_size: int, grid_pips: int, sl_grid_count: int, 
                stop_loss: str, margin_sl_percent: float, sizing: str, inputs_list: list, inputs_file: str):
    sim_name = f'{ticker}-{frequency}-{counter}'
    header = ['sim_name', 'init_trade_size', 'grid_pips', 'stop_loss', 'sl_grid_count', 'stoploss_pips', 'margin_sl_percent', 'sizing']
    inputs = [sim_name, init_trade_size, grid_pips, stop_loss, sl_grid_count, grid_pips * sl_grid_count, margin_sl_percent, sizing]
    print(tabulate([inputs], header, tablefmt='plain'))
    result= run_sim(
        d=d,
        sim_name=sim_name,
        start=start,
        end=end,
        init_bal=init_bal,
        init_trade_size=init_trade_size,
        grid_pips=grid_pips,
        sl_grid_count=sl_grid_count,
        stop_loss=stop_loss,
        margin_sl_percent=margin_sl_percent,
        sizing=sizing
    )
    # print('Saving files...')
    # with open(f'D:/Trading/ml4t-data/grid/{sim_name}.pkl', 'wb') as file:
    #     pkl.dump(result, file)
    result['result'].to_csv(f'D:/Trading/ml4t-data/grid/{sim_name}.csv', index=False)

    inputs_list.append(inputs)
    inputs_df = pd.DataFrame(inputs_list, columns=header)
    # inputs_df.to_pickle(f'D:/Trading/ml4t-data/grid/{ticker}-{frequency}-' + inputs_file)
    inputs_df.to_csv(f'D:/Trading/ml4t-data/grid/{ticker}-{frequency}-' + inputs_file, index=False)
    
    # del d.df[sim_name]
    counter =  counter + 1
    return inputs

In [22]:
# tickers = ['EUR_USD']
# frequency = ['M5']
# init_bal = [500, 1000, 2000]
# trade_size = [10, 100, 1000]
# grid_pips = [10, 20, 30, 40, 50, 60, 100]
# stop_loss = ['grid_count', 'margin_closeout_oldest', 'margin_closeout_farthest', 'margin_closeout_oldest_1', 'margin_closeout_farthest_1']
# sl_grid_count = [5, 10, 15, 20, 30, 50, 100, 10000]
# margin_sl_percent = [0.90, 0.80, 0.70, 0.60]
# sizing = ['static', 'dynamic']
# INPUTS_FILE = 'inputs.pkl'

In [23]:
checkpoint = 1
counter = 2001
start = None
end = None
tickers = ['EUR_USD']
frequency = ['M5']
init_bal = [1000]
init_trade_size = [1000]
grid_pips = [10, 20, 30, 40, 50, 100]
sl_grid_count = [5, 10, 15, 20, 30, 50, 100, 10000]
stop_loss = ['margin_closeout_oldest', 'margin_closeout_farthest'] # , 'margin_closeout_oldest_1', 'margin_closeout_farthest_1']
margin_sl_percent = [0.90, 0.80, 0.70]
sizing = ['dynamic', 'static']
INPUTS_FILE = 'inputs.2.csv'

In [24]:
# checkpoint = 1
# counter = 1001
# start = None
# end = None
# tickers = ['EUR_USD']
# frequency = ['M1']
# init_bal = [1000]
# init_trade_size = [1000]
# grid_pips = [10]
# sl_grid_count = [5]
# stop_loss = ['grid_count']
# margin_sl_percent = [0.90]
# sizing = ['dynamic']
# INPUTS_FILE = 'inputs.1049.csv'

In [25]:
inputs_list = list()
for tk in tickers:
    for f in frequency:
        df = read_data(tk, f)
        for ib in init_bal:
            for t in init_trade_size:
                for s in sizing:
                    for g in grid_pips:
                        for sl in stop_loss:
                            if sl == 'grid_count':
                                for slgc in sl_grid_count:
                                    if counter >= checkpoint:
                                        d = Data(df)
                                        inputs = process_sim(
                                            d=d,
                                            ticker=tk,
                                            frequency=f,
                                            counter=counter,
                                            start=start,
                                            end=end,
                                            init_bal=ib,
                                            init_trade_size=t,
                                            grid_pips=g,
                                            sl_grid_count=slgc,
                                            stop_loss=sl,
                                            margin_sl_percent=None,
                                            sizing=s,
                                            inputs_list=inputs_list,
                                            inputs_file=INPUTS_FILE
                                        )  
                                    counter =  counter + 1
                            else:
                                for mslp in margin_sl_percent:
                                    if counter >= checkpoint:
                                        d = Data(df)
                                        inputs = process_sim(
                                            d=d,
                                            ticker=tk,
                                            frequency=f,
                                            counter=counter,
                                            start=start,
                                            end=end,
                                            init_bal=ib,
                                            init_trade_size=t,
                                            grid_pips=g,
                                            stop_loss=sl,
                                            sl_grid_count=sl_grid_count[0],
                                            margin_sl_percent=mslp,
                                            sizing=s,
                                            inputs_list=inputs_list,
                                            inputs_file=INPUTS_FILE
                                        )  
                                    counter =  counter + 1

sim_name           init_trade_size    grid_pips  stop_loss                 sl_grid_count    stoploss_pips    margin_sl_percent  sizing
EUR_USD-M5-2001               1000           10  margin_closeout_oldest                5               50                  0.9  dynamic


 Simulating... : 100%|██████████| 594303/594303 [04:15<00:00, 2327.94it/s]


sim_name           init_trade_size    grid_pips  stop_loss                 sl_grid_count    stoploss_pips    margin_sl_percent  sizing
EUR_USD-M5-2002               1000           10  margin_closeout_oldest                5               50                  0.8  dynamic


 Simulating... : 100%|██████████| 594303/594303 [05:24<00:00, 1832.04it/s]


sim_name           init_trade_size    grid_pips  stop_loss                 sl_grid_count    stoploss_pips    margin_sl_percent  sizing
EUR_USD-M5-2003               1000           10  margin_closeout_oldest                5               50                  0.7  dynamic


...

 Simulating... : 100%|██████████| 594303/594303 [01:57<00:00, 5038.63it/s]


sim_name           init_trade_size    grid_pips  stop_loss                   sl_grid_count    stoploss_pips    margin_sl_percent  sizing
EUR_USD-M5-2071               1000          100  margin_closeout_farthest                5              500                  0.8  static


 Simulating... : 100%|██████████| 594303/594303 [01:54<00:00, 5168.64it/s]


sim_name           init_trade_size    grid_pips  stop_loss                   sl_grid_count    stoploss_pips    margin_sl_percent  sizing
EUR_USD-M5-2072               1000          100  margin_closeout_farthest                5              500                  0.7  static


 Simulating... : 100%|██████████| 594303/594303 [01:54<00:00, 5206.24it/s]


In [26]:
stop error

SyntaxError: invalid syntax (4267763832.py, line 1)

In [ ]:
i1 = pd.read_csv(f'D:/Trading/ml4t-data/grid/EUR_USD-M5-inputs.1.csv')
i2 = pd.read_csv(f'D:/Trading/ml4t-data/grid/EUR_USD-M5-inputs.2.csv')
i3 = pd.read_csv(f'D:/Trading/ml4t-data/grid/EUR_USD-M5-inputs.3.csv')
inputs = pd.concat([i1, i2, i3])

In [ ]:
# inputs = pd.read_csv(f'D:/Trading/ml4t-data/grid/EUR_USD-M5-inputs.1.csv')

In [ ]:
results = list()
for index, row in inputs.iterrows():
    result = pd.read_csv(f'D:/Trading/ml4t-data/grid/{row["sim_name"]}.csv')
    result['margin_ratio'] = (result['margin_closeout'] / result['margin_used']).round(2)
    header = list(inputs.columns) + ['ac_bal', 'min_margin_ratio', 'bars']
    # print(type(row), type(results['ac_bal'].iloc[-1]))
    results.append(list(row) + [result['ac_bal'].iloc[-1], result['margin_ratio'].min(), result.shape[0]])
print(tabulate(results, header, tablefmt='plain'))
# print(type(row.values), results['ac_bal'].iloc[-1])


In [ ]:
results = pd.DataFrame(results, columns=header)
results.to_csv('D:/Trading/ml4t-data/grid/EUR_USD-M1-1049-results.csv')

In [ ]:
results[results.min_margin_ratio > 0].sort_values(by=['ac_bal'], ascending=False)

In [ ]:
stop error

In [ ]:
# results = pd.read_pickle(f'D:/Trading/ml4t-data/grid/EUR_USD-M5-3.pkl')
results = pd.read_csv(f'D:/Trading/ml4t-data/grid/EUR_USD-M1-1001.csv')

In [ ]:
results

In [ ]:
# results = results['results'].copy()

In [ ]:
# results = results[results['trade']==True]

In [ ]:
results['net_bal'] = results['ac_bal'] + results['unrealised_pnl']
results['cum_position'] = results['cum_long_position'] + results['cum_short_position']


In [ ]:
results

In [ ]:
results.info()

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['mid_c'])

In [ ]:
cp = CandlePlot(results, candles=False, log=True)
cp.show_plot(line_traces=['ac_bal', 'net_bal', 'unrealised_pnl'])

In [ ]:
cp = CandlePlot(results, candles=False, log=True)
cp.show_plot(line_traces=['margin_used'])

In [ ]:
cp = CandlePlot(results, candles=False, log=True)
cp.show_plot(line_traces=['cum_long_position', 'cum_short_position'])

In [ ]:
results['margin_used_50'] = results['margin_used'] * 0.5
results['margin_call'] = results['margin_used_50'] > results['margin_closeout']
results['margin_ratio'] = (results['margin_closeout'] / results['margin_used']).round(2)

In [ ]:
cp = CandlePlot(results, candles=False, log=True)
cp.show_plot(line_traces=['margin_ratio', 'margin_call'])

In [ ]:
cp = CandlePlot(results, candles=False)
cp.show_plot(line_traces=['margin_call'])

In [ ]:
cp = CandlePlot(results, candles=False, log=True)
cp.show_plot(line_traces=['ac_bal'])